In [1]:
import numpy as np
import pandas as pd

# Loading Data

In [6]:

stop_data = pd.read_csv('./data/2015_Austin_traffic_stops.csv', parse_dates=[1,4])

# Row Processing

In [29]:
stop_data.columns

Index([u'HA_Arrest_Key', u'arrest_date', u'HA_YEAR', u'HA_MONTH',
       u'birth_date', u'HA_DAY_OF_WEEK', u'HA_SERVICE', u'officer_id',
       u'HA_Race_Sex', u'HA_Road_Class', u'HA_VEH_SEARCH', u'HA_ACCIDENT',
       u'HA_REASON_WARN', u'HA_REASON_CITA', u'HA_SEARCHED', u'HA_VEH_COLOR',
       u'HA_VEH_YEAR', u'HA_VEH_MODEL', u'HA_VEH_MAKE', u'race', u'sex',
       u'age_at_time_of_stop'],
      dtype='object')

In [24]:
stop_data['race'] = stop_data.HA_Race_Sex.apply(lambda x: x[0])
stop_data['sex'] = stop_data.HA_Race_Sex.apply(lambda x: x[1])
stop_data['age_at_time_of_stop'] = (stop_data['arrest_date'] - stop_data['birth_date']).apply(lambda x: (x / np.timedelta64(365, 'D')).astype(int))

# Group by day-of-week

In [28]:
stops_by_day_of_week = stop_data.groupby(["HA_DAY_OF_WEEK"]).count()

# Group by race and search

In [34]:
stops_by_race_search = stop_data.groupby(['race','HA_SEARCHED'],as_index=False).agg({'HA_Arrest_Key':lambda x: len(x.unique())})

# Selecting, renaming, and dropping

In [49]:
searched = stops_by_race_search[stops_by_race_search.HA_SEARCHED == 1].rename(columns={'HA_Arrest_Key':'searched_count'}).drop('HA_SEARCHED',axis=1)
nonsearched = stops_by_race_search[stops_by_race_search.HA_SEARCHED == 0].rename(columns={'HA_Arrest_Key':'nonsearched_count'}).drop('HA_SEARCHED',axis=1)

# Comparing search rates

In [55]:
search_by_race = pd.merge(nonsearched, searched, on='race',how='inner')
search_by_race['total'] = search_by_race['nonsearched_count'] + search_by_race['searched_count']
search_by_race['search_percentage'] = search_by_race['searched_count'] / search_by_race['total']

In [56]:
search_by_race

,race,nonsearched_count,searched_count,total,search_percentage
0,B,527,6,533,0.011257
1,H,822,28,850,0.032941
2,U,353,7,360,0.019444
3,W,3127,27,3154,0.008561
